In [1]:
# Import Dependencies
import copy
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

## Define a Single Layer Neural Network

In [2]:
class Net(nn.Module):
    def __init__(self, in_dim, out_dim):
        super(Net, self).__init__()
        
        self.in_dim = in_dim
        self.out_dim = out_dim
        
        self.layer = nn.Linear(in_features=self.in_dim, out_features=self.out_dim)
    
    def forward(self, x):
        out = self.layer(x)
        
        return out

In [3]:
# Instantiate the Model and print it out
model = Net(in_dim=1, out_dim=1)
print(model)

Net(
  (layer): Linear(in_features=1, out_features=1, bias=True)
)


In [4]:
# Create two copies of the same model
model_huber_wrapper = copy.deepcopy(model)
model_huber_class = copy.deepcopy(model)

## Create the Dataset Class

Creating the dataset class is optional to load the dattaset. Instead you can do this as well:

```
x = np.array([-1.0, 0.0, 1.0, 2.0, 3.0, 4.0], dtype=np.float32)
y = np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0], dtype=np.float32)
```

and then in the training loop, pass the dataset like following:

```
inputs = Variable(torch.from_numpy(x))
labels = Variable(torch.from_numpy(y))
```

This way you don't need to create the dataloader to load the dataset.

In [5]:
# Creating the artificial dataset
class SampleDataset(Dataset):
    # Constructor
    def __init__(self):
        x = np.array([-1.0, 0.0, 1.0, 2.0, 3.0, 4.0], dtype=np.float32)
        y = np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0], dtype=np.float32)
        self.xs = torch.from_numpy(x)
        self.ys = torch.from_numpy(y)
    
    # Getter
    def __getitem__(self, index):
        return self.xs[index], self.ys[index]
    
    # Getting the length
    def __len__(self):
        return len(self.xs)

In [6]:
dataset = SampleDataset()
trainloader = DataLoader(dataset=dataset)

In [7]:
# Model optimizer - SGD
optimizer = torch.optim.SGD(model_huber_wrapper.parameters(), lr=0.001)

## Custom Loss with Hyperparameter

In [8]:
def my_huber_loss_with_threshold(threshold):
    
    def huber_loss(y_true, y_pred):
        error = y_true - y_pred
        is_small_error = torch.abs(error) <= threshold
        small_error_loss = torch.square(error) / 2
        big_error_loss = threshold * (torch.abs(error) - (0.5 * threshold))

        return torch.where(is_small_error, small_error_loss, big_error_loss)
    
    return huber_loss

## Training the Model with Huber Loss Criterion

Note that an alternative way of using the Huber loss function could be as follows:

```
criterion = my_huber_loss_with_threshold(threshold=1.2)
loss = criterion(y, output)
```

instead of using

```
loss = my_huber_loss_with_threshold(threshold=1.2)(y, output)
```

In [9]:
EPOCHS = 500

for epoch in tqdm(range(EPOCHS)):
    running_loss = 0
    for idx, (x, y) in enumerate(trainloader):
        optimizer.zero_grad()
        output = model_huber_wrapper(x)
        loss = my_huber_loss_with_threshold(threshold=1.2)(y, output)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch: {epoch}, Loss: {running_loss/len(trainloader)}")

  8%|█████▋                                                                 | 40/500 [00:00<00:01, 395.00it/s]

Epoch: 0, Loss: 1.6339493294556935
Epoch: 1, Loss: 1.606069785853227
Epoch: 2, Loss: 1.5782703844209511
Epoch: 3, Loss: 1.5505507563551266
Epoch: 4, Loss: 1.5229105614125729
Epoch: 5, Loss: 1.4953493848443031
Epoch: 6, Loss: 1.4678669460117817
Epoch: 7, Loss: 1.4404627978801727
Epoch: 8, Loss: 1.4131364561617374
Epoch: 9, Loss: 1.3858876166244347
Epoch: 10, Loss: 1.3587162494659424
Epoch: 11, Loss: 1.3316218592226505
Epoch: 12, Loss: 1.3046038908263047
Epoch: 13, Loss: 1.2776619084179401
Epoch: 14, Loss: 1.2507958561182022
Epoch: 15, Loss: 1.2240054694314797
Epoch: 16, Loss: 1.1972900989154975
Epoch: 17, Loss: 1.1706494676570098
Epoch: 18, Loss: 1.144083421677351
Epoch: 19, Loss: 1.1175912705560525
Epoch: 20, Loss: 1.0911728801826637
Epoch: 21, Loss: 1.0648279475669067
Epoch: 22, Loss: 1.038577872638901
Epoch: 23, Loss: 1.0125776268541813
Epoch: 24, Loss: 0.9868877567350864
Epoch: 25, Loss: 0.9615036727239689
Epoch: 26, Loss: 0.9364211782813072
Epoch: 27, Loss: 0.9116363941381375
Epoch

 17%|████████████▏                                                          | 86/500 [00:00<00:00, 432.14it/s]

Epoch: 58, Loss: 0.30378088673266274
Epoch: 59, Loss: 0.2910049622878432
Epoch: 60, Loss: 0.2786892122433831
Epoch: 61, Loss: 0.2670985412163039
Epoch: 62, Loss: 0.2562309722416103
Epoch: 63, Loss: 0.24603903371219835
Epoch: 64, Loss: 0.23647826506445804
Epoch: 65, Loss: 0.22750703136747083
Epoch: 66, Loss: 0.2190870220462481
Epoch: 67, Loss: 0.2111822096630931
Epoch: 68, Loss: 0.20375873614102602
Epoch: 69, Loss: 0.19678500729302564
Epoch: 70, Loss: 0.19023160139719644
Epoch: 71, Loss: 0.18407112453132868
Epoch: 72, Loss: 0.17827776353806257
Epoch: 73, Loss: 0.17282780803119144
Epoch: 74, Loss: 0.16769847560984394
Epoch: 75, Loss: 0.16286894058187804
Epoch: 76, Loss: 0.15831977284202972
Epoch: 77, Loss: 0.15403265591400364
Epoch: 78, Loss: 0.14999045159978172
Epoch: 79, Loss: 0.14617744142500064
Epoch: 80, Loss: 0.14257854527871436
Epoch: 81, Loss: 0.13917990197660401
Epoch: 82, Loss: 0.1359685974312015
Epoch: 83, Loss: 0.13293234006656954
Epoch: 84, Loss: 0.13005990415695123
Epoch: 8

 27%|██████████████████▌                                                   | 133/500 [00:00<00:00, 447.32it/s]

...

Epoch: 335, Loss: 0.020695987666840665
Epoch: 336, Loss: 0.020568038132599515
Epoch: 337, Loss: 0.02044089242796569
Epoch: 338, Loss: 0.020314528007171855
Epoch: 339, Loss: 0.02018896247439746
Epoch: 340, Loss: 0.02006416606066826
Epoch: 341, Loss: 0.019940135589422425
Epoch: 342, Loss: 0.019816869355660554
Epoch: 343, Loss: 0.019694370257639093
Epoch: 344, Loss: 0.019572619005581753
Epoch: 345, Loss: 0.019451626064740896
Epoch: 346, Loss: 0.01933138169988524
Epoch: 347, Loss: 0.01921187448897399
Epoch: 348, Loss: 0.01909311511735723
Epoch: 349, Loss: 0.018975067800662753
Epoch: 350, Loss: 0.01885776781394573
Epoch: 351, Loss: 0.018741175459581427
Epoch: 352, Loss: 0.01862533894806499
Epoch: 353, Loss: 0.018510179867007537
Epoch: 354, Loss: 0.01839575925684282
Epoch: 355, Loss: 0.018282024466316216
Epoch: 356, Loss: 0.01816903334596039
Epoch: 357, Loss: 0.01805671012940972
Epoch: 358, Loss: 0.017945077971186645
Epoch: 359, Loss: 0.01783414834062569
Epoch: 360, Loss: 0.01772389713732991

 84%|██████████████████████████████████████████████████████████▌           | 418/500 [00:00<00:00, 459.85it/s]

Epoch: 370, Loss: 0.016658250662355083
Epoch: 371, Loss: 0.016555276098188187
Epoch: 372, Loss: 0.01645294101975499
Epoch: 373, Loss: 0.01635123473048831
Epoch: 374, Loss: 0.016250155842499225
Epoch: 375, Loss: 0.016149697682218783
Epoch: 376, Loss: 0.01604986240636208
Epoch: 377, Loss: 0.015950649562000763
Epoch: 378, Loss: 0.015852046352241207
Epoch: 379, Loss: 0.015754052554560378
Epoch: 380, Loss: 0.015656668218374154
Epoch: 381, Loss: 0.015559868067612115
Epoch: 382, Loss: 0.015463678115641718
Epoch: 383, Loss: 0.015368087312405502
Epoch: 384, Loss: 0.015273097323188267
Epoch: 385, Loss: 0.015178666280614076
Epoch: 386, Loss: 0.015084842974526206
Epoch: 387, Loss: 0.014991580533205706
Epoch: 388, Loss: 0.014898922525996264
Epoch: 389, Loss: 0.014806807835460253
Epoch: 390, Loss: 0.014715276624580534
Epoch: 391, Loss: 0.014624322024853123
Epoch: 392, Loss: 0.014533933213897399
Epoch: 393, Loss: 0.014444090394439021
Epoch: 394, Loss: 0.014354805668517656
Epoch: 395, Loss: 0.01426604

100%|██████████████████████████████████████████████████████████████████████| 500/500 [00:01<00:00, 454.35it/s]

Epoch: 461, Loss: 0.009474641708341855
Epoch: 462, Loss: 0.00941606075502932
Epoch: 463, Loss: 0.009357853918421219
Epoch: 464, Loss: 0.009299997646545913
Epoch: 465, Loss: 0.0092425212718202
Epoch: 466, Loss: 0.009185383151816495
Epoch: 467, Loss: 0.009128594834085865
Epoch: 468, Loss: 0.009072168027159933
Epoch: 469, Loss: 0.009016087105919723
Epoch: 470, Loss: 0.008960353747170302
Epoch: 471, Loss: 0.008904965281847884
Epoch: 472, Loss: 0.008849925371881303
Epoch: 473, Loss: 0.008795217614533613
Epoch: 474, Loss: 0.008740841464411156
Epoch: 475, Loss: 0.008686803040897454
Epoch: 476, Loss: 0.00863310679415008
Epoch: 477, Loss: 0.008579743505227574
Epoch: 478, Loss: 0.008526703689919183
Epoch: 479, Loss: 0.008473991529475219
Epoch: 480, Loss: 0.008421597842546666
Epoch: 481, Loss: 0.008369539119788291
Epoch: 482, Loss: 0.00831780754894377
Epoch: 483, Loss: 0.00826638388025458
Epoch: 484, Loss: 0.008215287124130555
Epoch: 485, Loss: 0.008164494018880456
Epoch: 486, Loss: 0.00811401036

## Inference using Huber Loss Wrapper Trained Model

In [10]:
sample = [10.0]

# Set model to Eval mode
model_huber_wrapper.eval()
output = model_huber_wrapper(torch.tensor(sample))
print(output)

tensor([18.6420], grad_fn=<AddBackward0>)


## Define Custom Loss Function

Here, we define the Huber Loss function that takes in two parameters:

```
y_true: the actual label
y_pred: the predicted output by the model
```

In [11]:
class HuberLoss(nn.Module):
    def __init__(self, threshold):
        super(HuberLoss, self).__init__()
        self.threshold = threshold
    
    def forward(self, y_true, y_pred):
        error = y_true - y_pred
        is_small_error = torch.abs(error) <= self.threshold
        small_error_loss = torch.square(error) / 2
        big_error_loss = self.threshold * (torch.abs(error) - (0.5 * self.threshold))
        
        return torch.where(is_small_error, small_error_loss, big_error_loss)

In [12]:
# Model optimizer - SGD
optimizer = torch.optim.SGD(model_huber_class.parameters(), lr=0.001)

## Train the Model with Huber Loss Function

Note that an alternative way of using the Huber loss function could be as follows:

```
criterion = HuberLoss(threshold=1.2)
loss = criterion(y, output)
```

instead of using

```
loss = HuberLoss(threshold=1.02)(y, output)
```

In [13]:
EPOCHS = 500

for epoch in tqdm(range(EPOCHS)):
    running_loss = 0
    for idx, (x, y) in enumerate(trainloader):
        optimizer.zero_grad()
        output = model_huber_class(x)
        loss = HuberLoss(threshold=1.02)(y, output)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch: {epoch}, Loss: {running_loss/len(trainloader)}")

  0%|                                                                                 | 0/500 [00:00<?, ?it/s]

Epoch: 0, Loss: 1.4593588970601559
Epoch: 1, Loss: 1.4388243469099204
Epoch: 2, Loss: 1.4183488003909588
Epoch: 3, Loss: 1.39793220286568
Epoch: 4, Loss: 1.377574160695076
Epoch: 5, Loss: 1.3572744131088257
Epoch: 6, Loss: 1.3370326136549313
Epoch: 7, Loss: 1.3168485711018245
Epoch: 8, Loss: 1.2967220780750115
Epoch: 9, Loss: 1.276652862628301
Epoch: 10, Loss: 1.2566406304637592
Epoch: 11, Loss: 1.236685066173474
Epoch: 12, Loss: 1.2167858531077702
Epoch: 13, Loss: 1.196943011134863
Epoch: 14, Loss: 1.1771560894946258
Epoch: 15, Loss: 1.157424954076608
Epoch: 16, Loss: 1.137749249736468
Epoch: 17, Loss: 1.1181286126375198
Epoch: 18, Loss: 1.0985628354052703
Epoch: 19, Loss: 1.0790517355004947
Epoch: 20, Loss: 1.0595950732628505
Epoch: 21, Loss: 1.0401926959554355
Epoch: 22, Loss: 1.0208442422250907
Epoch: 23, Loss: 1.0015493494768937
Epoch: 24, Loss: 0.9823079208532969
Epoch: 25, Loss: 0.9631197775403658
Epoch: 26, Loss: 0.9439844905088345
Epoch: 27, Loss: 0.9249021125336488
Epoch: 28,

  8%|█████▌                                                                 | 39/500 [00:00<00:01, 382.53it/s]

Epoch: 34, Loss: 0.7928392638762792
Epoch: 35, Loss: 0.7743846823771795
Epoch: 36, Loss: 0.7561482808863124
Epoch: 37, Loss: 0.7381265467653672
Epoch: 38, Loss: 0.7203166745603085
Epoch: 39, Loss: 0.7027159774055084
Epoch: 40, Loss: 0.6853211068858703
Epoch: 41, Loss: 0.6681293387276431
Epoch: 42, Loss: 0.6511379952232043
Epoch: 43, Loss: 0.6343440939672291
Epoch: 44, Loss: 0.6177450804971159
Epoch: 45, Loss: 0.6013381453230977
Epoch: 46, Loss: 0.5851207891634355
Epoch: 47, Loss: 0.5690901086976131
Epoch: 48, Loss: 0.5532437614165246
Epoch: 49, Loss: 0.5375790335626031
Epoch: 50, Loss: 0.5220934509610137
Epoch: 51, Loss: 0.5067846482464423
Epoch: 52, Loss: 0.491649835263767
Epoch: 53, Loss: 0.47668695011816453
Epoch: 54, Loss: 0.46189354769982555
Epoch: 55, Loss: 0.44726707096318324
Epoch: 56, Loss: 0.43280550051808103
Epoch: 57, Loss: 0.41850618311097304
Epoch: 58, Loss: 0.4043672705253509
Epoch: 59, Loss: 0.39048017729267787
Epoch: 60, Loss: 0.37699284737270017
Epoch: 61, Loss: 0.363

 16%|███████████                                                            | 78/500 [00:00<00:01, 378.19it/s]

...

Epoch: 411, Loss: 0.013575445088160146
Epoch: 412, Loss: 0.013491518728793986
Epoch: 413, Loss: 0.013408111193105773
Epoch: 414, Loss: 0.013325234510072429
Epoch: 415, Loss: 0.013242857916338835
Epoch: 416, Loss: 0.013160990000869788
Epoch: 417, Loss: 0.013079631501871821
Epoch: 418, Loss: 0.012998783296097827
Epoch: 419, Loss: 0.012918429875753645
Epoch: 420, Loss: 0.012838569702580571
Epoch: 421, Loss: 0.012759212226228556
Epoch: 422, Loss: 0.012680345361635167
Epoch: 423, Loss: 0.012601958432848429
Epoch: 424, Loss: 0.012524044857248859
Epoch: 425, Loss: 0.012446626291421126
Epoch: 426, Loss: 0.012369684972327377
Epoch: 427, Loss: 0.012293224588574958
Epoch: 428, Loss: 0.012217239908750344
Epoch: 429, Loss: 0.012141706723317233
Epoch: 430, Loss: 0.012066643703292357
Epoch: 431, Loss: 0.011992058023376254
Epoch: 432, Loss: 0.011917929204476726
Epoch: 433, Loss: 0.011844234489520508
Epoch: 434, Loss: 0.011771012429259523
Epoch: 435, Loss: 0.011698245173344427
Epoch: 436, Loss: 0.01162

 92%|████████████████████████████████████████████████████████████████▎     | 459/500 [00:01<00:00, 420.46it/s]

Epoch: 449, Loss: 0.01072547456836522
Epoch: 450, Loss: 0.010659190927981399
Epoch: 451, Loss: 0.010593295351706425
Epoch: 452, Loss: 0.01052780340736111
Epoch: 453, Loss: 0.010462718696847636
Epoch: 454, Loss: 0.010398048330292417
Epoch: 455, Loss: 0.010333771427819253
Epoch: 456, Loss: 0.010269880663448324
Epoch: 457, Loss: 0.010206394364407364
Epoch: 458, Loss: 0.010143305216843146
Epoch: 459, Loss: 0.010080605442453816
Epoch: 460, Loss: 0.010018276084641306
Epoch: 461, Loss: 0.009956359648034171
Epoch: 462, Loss: 0.009894811025333183
Epoch: 463, Loss: 0.009833648997603936
Epoch: 464, Loss: 0.009772846828733842
Epoch: 465, Loss: 0.009712448103528004
Epoch: 466, Loss: 0.009652399236074416
Epoch: 467, Loss: 0.009592730933945859
Epoch: 468, Loss: 0.009533436703956491
Epoch: 469, Loss: 0.009474495508584369
Epoch: 470, Loss: 0.009415926993824542
Epoch: 471, Loss: 0.009357715948378123
Epoch: 472, Loss: 0.009299873974669026
Epoch: 473, Loss: 0.009242390416754157
Epoch: 474, Loss: 0.0091852

100%|██████████████████████████████████████████████████████████████████████| 500/500 [00:01<00:00, 404.94it/s]

Epoch: 493, Loss: 0.008164369260460566
Epoch: 494, Loss: 0.008113897166670844
Epoch: 495, Loss: 0.008063736332284558
Epoch: 496, Loss: 0.008013883606812064
Epoch: 497, Loss: 0.007964354461212983
Epoch: 498, Loss: 0.007915117273417612
Epoch: 499, Loss: 0.007866198451665696


## Inference using Huber Loss Trained Model

In [14]:
sample = [10.0]

# Set model to Eval mode
model_huber_class.eval()
output = model_huber_class(torch.tensor(sample))
print(output)

tensor([18.6330], grad_fn=<AddBackward0>)
